In [2]:
import torch
import diopilib
from conformance.diopi_functions import spmm,spconv
from conformance.diopi_runtime import Tensor
import numpy as np
# from torch.nn.functional import dropout
# sparse matrix:
# 0.5,  0,  1
# 0,    0,  2
# 1,    3,  0.6

# M, K, N = 4096, 4096, 128

# a = torch.randn((M,K),dtype=torch.float32)
# a = dropout(a, p=0.9)
# sparse_a = a.to_sparse_csr()
# print(a)
# print(sparse_a)
# b = np.random.randn(K,N).astype(np.float32)
# print(b)

# input = Tensor.from_numpy(b)
# row_ptr = Tensor.from_numpy(sparse_a.crow_indices().numpy().astype(np.int32))
# col_ind = Tensor.from_numpy(sparse_a.col_indices().numpy().astype(np.int32))
# values = Tensor.from_numpy(sparse_a.values().numpy().astype(np.float32))
# print(list(input.size().data))
# c = spmm(row_ptr, col_ind, values, input)
# c_ref = a @ torch.from_numpy(b)
# # c = rbrmsr_spmm(sparse_a.crow_indices(), sparse_a.col_indices(), sparse_a.values(), b)


spconv_data = torch.load('spconv_data.pth')
in_feat = spconv_data['in_feats']
kernel = spconv_data['kernel']
neighbor_map = spconv_data['neighbor_map']
sum_nnz = spconv_data['sum_nnz']
neighbor_address = spconv_data['neighbor_address']
q_neighbor_address = spconv_data['q_neighbor_address']
output_size = spconv_data['output_size']
qsum_nnz = spconv_data['qsum_nnz'].item()

in_feat = Tensor.from_numpy(in_feat.numpy().astype(np.float16))
kernel = Tensor.from_numpy(kernel.numpy().astype(np.float16))
neighbor_map = Tensor.from_numpy(neighbor_map.numpy().astype(np.int32))
neighbor_address = Tensor.from_numpy(neighbor_address.numpy().astype(np.int32))
q_neighbor_address = Tensor.from_numpy(q_neighbor_address.numpy().astype(np.int32))

out = spconv(in_feat, kernel,neighbor_map,sum_nnz,neighbor_address,q_neighbor_address,output_size,
           qsum_nnz,False,True,True) 

print(out)

ModuleNotFoundError: No module named 'diopilib'

In [2]:
c_ours = torch.from_numpy(c.numpy())
print("ours:", c_ours)
print("ref: ", c_ref)
assert torch.allclose(c_ours, c_ref, rtol=1e-03, atol=1e-03)

ours: tensor([[ 1.9180e+02, -3.3436e+02, -2.5628e+02,  ..., -1.2209e+02,
          1.2449e+02,  8.9544e+00],
        [ 5.4242e+00, -9.1456e+01, -8.8789e+01,  ...,  2.9239e+02,
          5.9588e+02, -6.7169e+01],
        [ 9.6391e+01, -2.8461e+02, -8.9830e+01,  ...,  2.4198e+02,
          1.9330e+02, -2.7285e+02],
        ...,
        [-9.6388e+01, -1.8851e+02, -3.9614e+02,  ...,  3.3607e+02,
          9.8614e+01, -2.6860e+02],
        [-8.1585e+01, -1.1838e+02, -1.9467e+02,  ...,  3.6153e+02,
         -1.0657e+02, -3.8279e+02],
        [ 1.8823e+02, -3.1072e+02, -2.3838e+02,  ..., -3.6757e-01,
         -5.7795e+01, -1.2785e+02]])
ref:  tensor([[ 1.9180e+02, -3.3436e+02, -2.5628e+02,  ..., -1.2209e+02,
          1.2449e+02,  8.9544e+00],
        [ 5.4243e+00, -9.1456e+01, -8.8789e+01,  ...,  2.9239e+02,
          5.9588e+02, -6.7170e+01],
        [ 9.6391e+01, -2.8461e+02, -8.9830e+01,  ...,  2.4198e+02,
          1.9330e+02, -2.7285e+02],
        ...,
        [-9.6388e+01, -1.8851e+02,